In [1]:
import logging
logging.basicConfig(level=logging.ERROR)
# basicConfig: 这是logging模块中的一个函数，用于设置日志的基本参数。
# level=logging.ERROR: 这里设置了日志级别为ERROR。这意味着只有当错误发生时（级别为ERROR或更高级别），日志消息才会被记录。低于这个级别的消息（例如WARNING、INFO或DEBUG）将被忽略。

In [2]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

In [3]:
USER_NAME = "Person A"

In [4]:
import sys
sys.path.append('/home/kit/kit/donson_intern/langchain')
from ChatGLM3 import ChatGLM3
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
llm("你好")

'Hello! How can I assist you today?'

In [6]:
import math
import faiss
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
"""Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
def relevance_score_fn(score: float) -> float:
   
   return 1.0 - score / math.sqrt(2)

"""Create a new vector store retriever unique to the agent."""
def create_new_memory_retriever():
    # Define your embedding model
    #embeddings_model = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
    embeddings_model = LangchainEmbedding(embeddings)
    
    # Initialize the vectorstore as empty
    embedding_size = 512
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {}, relevance_score_fn=relevance_score_fn)
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)    
 

In [7]:
from GenerativeAgent import GenerativeAgent
from GenerativeAgentMemory import GenerativeAgentMemory
#创建第一个智能体，tommies
tommies_memory = GenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8 # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(name="Tommie", 
                         age=25,
                         traits="anxious, likes design, talkative", 
                         status="looking for a job", 
                         memory_retriever=create_new_memory_retriever(),
                         llm=llm,
                         memory=tommies_memory
                         )
 

In [8]:
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie\


In [9]:
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
 
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [10]:
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is feeling tired and hungry after a long journey. He sees a new home and tries to get some rest, but is interrupted by the noise of the road at night. He also remembers his dog, Bruno, and the new neighbors have a cat.


In [11]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    # 使用者和智能体交谈接口
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [12]:
new_message = "Person A says What do you like to do? "
tommie.generate_dialogue_response(new_message)[0]

True

In [13]:
interview_agent(tommie, "What do you like to do?")

"Tommie said TOMMIE: Hi there! It's nice to meet you. As for what I like to do, I enjoy designing and talking about various topics. What about you?"

In [14]:
interview_agent(tommie, "What are you looking forward to doing today?")

"Tommie said TOMMIE: Hi there! It's nice to meet you. As for what I'm looking forward to doing today, I'm actually looking for a job. I've been traveling a lot lately and I'm ready to settle down somewhere. What about you?"

In [15]:
interview_agent(tommie, "What are you most worried about today?")

"Tommie said TOMMIE: I'm not really worried about anything at the moment. I'm just looking forward to getting some rest and settling down in my new home. Is there anything else you'd like to talk about?"

In [16]:
# 增加更多的观察
observations = [
 
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
 
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
 
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
 
    "Tommie finally finds the filters and makes himself a cup of coffee.",
 
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
 
    "Tommie checks his email and sees that he has no job offers yet.",
 
    "Tommie spends some time updating his resume and cover letter.",
 
    "Tommie heads out to explore the city and look for job openings.",
 
    "Tommie sees a sign for a job fair and decides to attend.",
 
    "The line to get in is long, and Tommie has to wait for an hour.",
 
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
 
    "Tommie leaves the job fair feeling disappointed.",
 
    "Tommie stops by a local diner to grab some lunch.",
 
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
 
    "Tommie overhears a conversation at the next table about a job opening.",
 
    "Tommie asks the diners about the job opening and gets some information about the company.",
 
    "Tommie decides to apply for the job and sends his resume and cover letter.",
 
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
 
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
 
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
 
    "Tommie sees a group of people playing frisbee and decides to join in.",
 
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
 
    "Tommie goes back to his apartment to rest for a bit.",
 
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
 
    "Tommie starts to feel frustrated with his job search.",
 
    "Tommie calls his best friend to vent about his struggles.",
 
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
 
    "Tommie feels slightly better after talking to his friend.",
 
]
 
 
 

In [17]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
 
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i+1) % 20) == 0:
        print('***************************************************')
        print(colored(f"After {i+1} observations, Tommie's summary is:{tommie.get_summary(force_refresh=True)}", "blue"))
        print('***************************************************')
 
 
 

Tommie wakes up to the sound of a noisy construction site outside his window. TOMMIE: Oh, that's interesting. I wonder what they're building over there. Can you tell me more about it?
Tommie gets out of bed and heads to the kitchen to make himself some coffee. TOMMIE: Ah, coffee. My favorite. I'll have to make a pot.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie looks confused and starts to search through his moving boxes for the coffee filters.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie takes a sip of his coffee and continues to relax in his new home.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie looks disappointed and tastes the coffee again.
Tommie checks his email and sees that he has no job offers yet. Tommie looks discouraged and starts to feel overwhelmed by the job search.
Tommie spends some time updating his resume and cover letter. Tommie looks

In [18]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said looks interested and decides to engage in dialogue. He says, "Hi there! It\'s been a pretty good day so far. I went to a job fair and met with some potential employers. It was a long wait in line to get in, but I\'m still feeling hopeful."'

In [19]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said might say, "I\'m not really a fan of coffee. It\'s always a bit bitter for me."'

In [20]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said Observation: Person A says Tell me about your childhood dog!'

In [21]:
# 创建第二个角色 eves
eves_memory = GenerativeAgentMemory(
    
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5
)

eve = GenerativeAgent(name="Eve", 
                      age=34, 
                      traits="curious, helpful", 
                      status="N/A", 
                      llm=llm,
                      daily_summaries = [
                          ("Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie.")
                          ],
                    memory=eves_memory,
                    verbose=False
             )

In [22]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")

eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)
 

In [23]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Based on the given statements, Eve is a helpful coworker who assistes others, she is also a morning person who wakes up to alarms, she eats porridge for breakfast, she plays tennis with her friends, and she is aware of her colleagues' opinions.


In [24]:
interview_agent(eve, "How are you feeling about today?")

'Eve said It depends on the context and the situation. If the colleague who mentioned Tommie being hard to work with is someone that Eve doesn\'t get along with or has had negative experiences with, Eve might respond with something like "I\'m not a big fan of working with Tommie either. He can be a bit difficult to get along with at times." If the situation is more positive, Eve might respond with something like "I\'m feeling pretty good today. How about you?" or "I\'m looking forward to playing tennis with Xu later." Without more information, it\'s hard to give a specific answer.'

In [25]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said It depends on the context and the situation. If the colleague who mentioned Tommie being hard to work with is someone that Eve doesn\'t get along with or has had negative experiences with, Eve might respond with something like "I\'m not a big fan of working with Tommie either. He can be a bit difficult to get along with at times." If the situation is more positive, Eve might respond with something like "I\'m feeling pretty good today. How about you?" or "I\'m looking forward to playing tennis with Xu later." Without more information, it\'s hard to give a specific answer.'

In [26]:
interview_agent(eve, "Tommie is looking to find a job. What are are some things you'd like to ask him?")

'Eve said It depends on the situation. If Tommie is a colleague that Eve has had negative experiences with, Eve might respond with something like "I\'m not a big fan of working with Tommie either. He can be a bit difficult to get along with at times." If the situation is more positive, Eve might respond with something like "I\'m feeling pretty good today. How about you?" or "I\'m looking forward to playing tennis with Xu later." Without more information, it\'s hard to give a specific answer.'

In [27]:
interview_agent(eve, "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.")

'Eve said "I\'m sorry, I don\'t understand what you\'re asking. Could you please rephrase it or provide more information?"'

In [28]:
# 让智能体之间展开对话
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    _, observation = agents[1].generate_reaction(initial_observation) #谈话开始
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True   
        if break_dialogue:
            break
        turns += 1

In [29]:
agents = [tommie, eve]
run_conversation(agents, "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?")

Eve said "I'd be happy to chat with you about my experience. Let's see, I've been working for a few years now and have had a mix of positive and negative experiences. What are you specifically interested in hearing about?"
Tommie said looks interested and decides to engage in dialogue. He says, "Eve, I'd love to hear about your experience. What are some of the positive and negative experiences you've had in your job?"
Eve said "I'd be happy to chat with you about my experience. Let's see, I've had a mix of positive and negative experiences in my job. What are you specifically interested in hearing about?"
Tommie said says, "Thank you for sharing your experience with me, Eve. I appreciate your willingness to chat with me. I'm glad that you've had both positive and negative experiences in your job. It's always helpful to hear about different experiences from someone who has been through it before. If you ever need to talk about anything or have any advice for me, please don't hesitate to

Input length of input_ids is 9157, but `max_length` is set to 8192. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1123,0,0], thread: [64,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1123,0,0], thread: [65,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1123,0,0], thread: [66,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1123,0,0], thread: [67,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [1123,0,0], thread: [68,0,0] Assertion `-sizes[i] <= index

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 4608 n 9157 k 4096 mat1_ld 4096 mat2_ld 4096 result_ld 4608 abcType 2 computeType 68 scaleType 0